In [1]:
import numpy as np
from gurobipy import *

In [2]:
with open('UFL.dat') as f:
    dem = np.fromstring(f.readline(), sep=',\t')
    c = np.fromstring(f.readline(), sep=',\t')
    c = c.reshape(len(dem), len(c)//len(dem))
    f = np.fromstring(f.readline(), sep=',\t')

In [3]:
n_clients = dem.shape[0]
print('Demand vector:', dem.shape)
n_facilities = f.shape[0]
print('Fixed-cost vector:', f.shape)
print('Variable-cost matrix:', c.shape)

Demand vector: (45,)
Fixed-cost vector: (40,)
Variable-cost matrix: (45, 40)


In [4]:
# Create model instance
mip = Model('Branch-and-Cut')

# Add variables
x = mip.addVars(n_clients, n_facilities, obj=c, vtype=GRB.CONTINUOUS, name='x')
y = mip.addVars(n_facilities, obj=f, vtype=GRB.BINARY, name='y')
mip.update()

# Add constraints
mip.addConstrs((x.sum(i, '*') == dem[i] for i in range(n_clients)), name='assignment_constr')
mip.addConstrs((x[i,j] <= dem[i]*y[j] for i in range(n_clients) for j in range(n_facilities)), name='disagg_capacity_constr')

# Set objective
mip.ModelSense = GRB.MINIMIZE

# Solve model
mip.optimize()

Academic license - for non-commercial use only
Optimize a model with 1845 rows, 1840 columns and 5400 nonzeros
Variable types: 1800 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+02]
Found heuristic solution: objective 1356880.0000
Presolve time: 0.01s
Presolved: 1845 rows, 1840 columns, 5400 nonzeros
Variable types: 1800 continuous, 40 integer (40 binary)

Root relaxation: objective 1.829190e+05, 105 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    182919.00000 182919.000  0.00%     -    0s

Explored 0 nodes (105 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 182919 1.35688e+06 

Optimal solution found (tolerance 1.00e-04)
Best obje

C:\Users\samarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """


In [5]:
# Write lp file
mip.write('mip.lp')